# Day 7: Hugging Face Transformers 基础（Pipeline、Tokenizer、Model）

本讲义介绍 Hugging Face `transformers` 库的三个核心组件，这是使用预训练 Transformer 模型的基础。

**学习目标：**
- 理解 Pipeline 的完整工作流（预处理 → 推理 → 后处理）
- 掌握 Tokenizer 的使用：分词、填充、截断
- 学习 AutoModel 的加载与前向传播
- 认识不同任务对应的 AutoModel 变体

**三个关键抽象：**
1. **Pipeline**：高层接口，一行代码解决任务（快速原型）
2. **Tokenizer**：文本处理器，将原始文本转换为模型输入
3. **Model**：神经网络，进行实际计算

**提示**：Pipeline 方便快速测试，但微调时通常需要直接使用 Tokenizer 和 Model。

## Pipeline：一行代码完成情感分析

**Pipeline** 是 Hugging Face 提供的最高层抽象，封装了整个推理流程：

**工作流程：**
```
用户输入文本 
    ↓
Tokenizer 分词 & 数值化
    ↓
Model 推理（获得原始输出 logits）
    ↓
后处理（softmax + 格式化）
    ↓
返回人类可读结果 {label, score}
```

**优点**：
- 零配置，开箱即用
- 自动选择合适的模型架构
- 支持批量输入

**缺点**：
- 功能有限（无法自定义损失、优化器等）
- 推理速度相对较慢（因为自动化带来的开销）
- 不适合微调任务

**使用场景**：快速原型、Demo、教学

In [1]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9598049521446228},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [ ]:
del classifier #防止占用内存

这里把整个模型都加载进了内存，进行推理。（67M，F32）       
查看模型相信信息。https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english

Pipeline的工作流程包含以下步骤：

根据输入的任务类型和 checkpoint，自动加载对应的 tokenizer 和 model

- 预处理 (Preprocessing):- 将输入文本转换为模型可以理解的格式- 进行分词 (tokenization)- 添加特殊标记（如[CLS], [SEP]等）- 将token转换为对应的ID
- 模型推理 (Model Inference):- 将处理后的输入传入预训练模型- 模型进行计算并输出原始预测结果
- 后处理 (Post-processing):- 将模型的原始输出转换为人类可理解的格式- 对结果进行格式化（如标签和置信度分数）

输入：

- 可以是单个文本字符串或文本列表
- 支持不同任务类型的特定输入格式

输出：

- 返回包含预测结果的字典或字典列表
- 结果通常包含：- label: 预测的标签- score: 预测的置信度分数（0-1之间）

## Tokenizer：将文本转换为数字

**Tokenizer** 的核心责任：
1. **分词（Tokenization）**：将文本拆分为 token（通常是子词）
2. **映射（Mapping）**：将 token 转换为对应的 ID
3. **填充与截断**：统一序列长度
4. **特殊标记**：添加 `[CLS]`, `[SEP]` 等模型特定的标记

**输出格式（字典）：**
- `input_ids`：token ID 序列
- `attention_mask`：1 表示有效 token，0 表示填充
- `token_type_ids`：用于句子对任务，区分两个句子

**关键参数：**
- `max_length`：截断或填充到指定长度
- `padding`：填充方式（`"max_length"`、`"longest"`）
- `truncation`：是否截断超长序列
- `return_tensors`：返回 PyTorch 张量（`"pt"`）还是 NumPy（`"np"`）

## AutoClass 的智能推断

**AutoClass（AutoTokenizer、AutoModel 等）如何工作？**

1. **根据 checkpoint 查询配置**：
   - 从 Hub 或本地路径下载 `config.json`
   - 读取 `model_type` 字段（如 `"bert"`、`"roberta"`）

2. **自动选择正确的类**：
   - `model_type="bert"` → `BertTokenizer` 或 `BertModel`
   - 无需手动指定类名，减少出错

3. **加载预训练权重**：
   - 自动下载并缓存模型权重
   - 一致性检查确保配置与权重匹配

**注意**：AutoClass 进行**精确匹配**，不做模糊匹配。
- ✅ `AutoModel.from_pretrained("bert-base-uncased")` → 成功
- ❌ `AutoModel.from_pretrained("ber-base-uncased")` → 错误（拼写错误）

In [ ]:
from transformers import AutoTokenizer
    
# 1. 加载 AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(
        "bert-base-uncased",        # checkpoint 名称
        use_fast=True,              # 使用快速分词器（推荐，基于 Rust 实现）
        add_prefix_space=False,     # 是否在开头添加空格（对某些模型如 RoBERTa 有用）
)
    
# 2. 输入文本
text = "Hello, how are you today? I am Grok, built by xAI!"
    
# 3. 使用 tokenizer 处理文本（设置常用参数）
encoded_output = tokenizer(
    text,                       # 输入文本（字符串或字符串列表）
    add_special_tokens=True,    # 是否添加特殊标记（如 [CLS], [SEP]）
    max_length=20,             # 最大序列长度（超过会截断）
    padding="max_length",       # 填充到 max_length（可选："longest", False 等）
    truncation=True,            # 超过 max_length 时截断
    return_tensors="pt",        # 返回 PyTorch 张量（可选："tf" 或 None）
    return_attention_mask=True, # 返回 attention mask
    return_token_type_ids=True, # 返回 token type IDs（用于区分句子对。在句子对任务中，第一个句子为 0，第二个句子为 1）
)
    
# 4. 输出结果 字典
print("原始文本:", text)
print("\nTokenizer 输出:")
print("input_ids:", encoded_output["input_ids"])
print("attention_mask:", encoded_output["attention_mask"])
print("token_type_ids:", encoded_output["token_type_ids"])
print("\n解码回文本:", tokenizer.decode(encoded_output["input_ids"][0]))
print("分词结果:", tokenizer.convert_ids_to_tokens(encoded_output["input_ids"][0]))
    
# 5. 额外功能：批量输入
batch_text = ["Hello world!", "I am Grok."]
batch_encoded = tokenizer(
    batch_text,
    padding=True,               # 自动填充到最长序列长度
    truncation=True,
    max_length=10,
    return_tensors="pt"
)
print("\n批量输入结果:")
print("input_ids:", batch_encoded["input_ids"])
print("attention_mask:", batch_encoded["attention_mask"])

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

原始文本: Hello, how are you today? I am Grok, built by xAI!

Tokenizer 输出:
input_ids: tensor([[  101,  7592,  1010,  2129,  2024,  2017,  2651,  1029,  1045,  2572,
         24665,  6559,  1010,  2328,  2011,  1060,  4886,   999,   102,     0]])
attention_mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])
token_type_ids: tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

解码回文本: [CLS] hello, how are you today? i am grok, built by xai! [SEP] [PAD]
分词结果: ['[CLS]', 'hello', ',', 'how', 'are', 'you', 'today', '?', 'i', 'am', 'gr', '##ok', ',', 'built', 'by', 'x', '##ai', '!', '[SEP]', '[PAD]']

批量输入结果:
input_ids: tensor([[  101,  7592,  2088,   999,   102,     0,     0],
        [  101,  1045,  2572, 24665,  6559,  1012,   102]])
attention_mask: tensor([[1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1]])


In [5]:
del tokenizer

**输入参数**
    
1. **from_pretrained() 参数**：
    - "bert-base-uncased": 指定预训练模型/checkpoint。
    - use_fast=True: 使用 Rust 实现的快速分词器，性能更好。
    - cache_dir: 指定缓存路径，避免重复下载。
    - add_prefix_space: 对某些模型（如 RoBERTa）有用，BERT 不需要。
2. **tokenizer() 参数**：
    - text: 输入可以是单个字符串或字符串列表。
    - add_special_tokens: 添加模型特定的标记（如 [CLS] 和 [SEP]）。
    - max_length: 限制序列长度。
    - padding: 填充方式（"max_length" 填充到指定长度，"longest" 填充到批次中最长序列）。
    - truncation: 超过 max_length 时截断。
    - return_tensors: 指定返回类型（"pt" 为 PyTorch，"tf" 为 TensorFlow，None 为 Python 列表）。
    - return_attention_mask: 返回注意力掩码，用于区分有效 token 和填充。
    - return_token_type_ids: 返回 token 类型 ID，用于句子对任务。
    
**输出内容**
    
1. **input_ids**：
    - 将文本转换为 token ID 的序列，每个 ID 对应词汇表中的一个 token。
    - [CLS]（101）和 [SEP]（102）是特殊标记，[PAD]（0）是填充。
2. **attention_mask**：
    - 二进制掩码，1 表示有效 token，0 表示填充。
    - 用于告诉模型哪些部分需要关注。
3. **token_type_ids**：
    - 用于区分不同句子（在单句输入中通常全为 0）。
    - 在句子对任务中，第一个句子为 0，第二个句子为 1。
4. **解码和分词**：
    - decode(): 将 input_ids 转换回可读文本。
    - convert_ids_to_tokens(): 显示具体的分词结果（如 "x" 和 "##ai" 是子词）。
    
**批量输入**
    
- 当输入是列表时，padding=True 会自动对齐序列长度，填充较短的句子。
- max_length 限制仍然有效。
    

**扩展功能**
    
1. **保存 tokenizer**：python
        
    ```python
    tokenizer.save_pretrained("./my_tokenizer")
    ```
        
2. **处理句子对**：python
        
    ```python
    encoded_pair = tokenizer("Hello!", "How are you?", return_tensors="pt")
    print(encoded_pair["token_type_ids"])  # 区分两个句子
    ```
        
3. **自定义词汇表**：python
        
    ```python
    tokenizer.add_tokens(["new_token"])
    ```

## Model：获取预训练特征表示

**AutoModel 与任务特定模型的区别：**

| 类 | 输出 | 用途 |
|------|------|------|
| `AutoModel` | `last_hidden_state`, `pooler_output` | 特征提取、迁移学习 |
| `AutoModelForSequenceClassification` | `logits` [batch, num_labels] | 文本分类 |
| `AutoModelForTokenClassification` | `logits` [batch, seq_len, num_labels] | NER、词性标注 |
| `AutoModelForQuestionAnswering` | `start_logits`, `end_logits` | 抽取式问答 |
| `AutoModelForCausalLM` | `logits` [batch, seq_len, vocab_size] | 文本生成（GPT 类） |

**重要参数：**
- `output_hidden_states=True`：返回所有层的隐藏状态（用于分析）
- `output_attentions=True`：返回注意力权重（用于可解释性分析）

**性能提示**：启用上述参数会**显著增加内存占用**。

In [3]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [6]:
del model

1. **加载模型**：
    - 根据 checkpoint 的配置文件，自动推断模型类型（如 BertModel）。
    - 使用 AutoModel.from_pretrained(checkpoint) 加载预训练模型。
2. **输入数据**：
    - AutoModel 需要接收 input_ids（来自 tokenizer 的输出）以及其他可选输入（如 attention_mask）。
    - 输入通常是张量（PyTorch 或 TensorFlow）。
3. **输出**：
    - 模型返回一个包含隐藏状态（hidden states）和其他输出的对象（具体取决于模型类型和配置）。
    - 不同任务的 AutoModel 变体返回值不同，需根据任务选择合适的类。
    - 默认 AutoModel 不包含头部（head），仅返回隐藏状态。

| **变体** | **主要输出字段** | **输出形状示例（BERT）** | **典型任务** |
| --- | --- | --- | --- |
| AutoModel | last_hidden_state, pooler_output | [1, 10, 768], [1, 768] | 特征提取 |
| AutoModelForSequenceClassification | logits | [1, 2] | 文本分类 |
| AutoModelForTokenClassification | logits | [1, 10, 3] | NER、词性标注 |
| AutoModelForQuestionAnswering | start_logits, end_logits | [1, 10], [1, 10] | 问答 |
| AutoModelForCausalLM | logits | [1, 10, 30522] | 文本生成（非 BERT） |
| AutoModelForMaskedLM | logits | [1, 10, 30522] | 掩码预测 |
| AutoModelForSeq2SeqLM | logits, encoder_last_hidden_state | [1, 10, vocab_size], [1, 10, 768] | 翻译、摘要（非 BERT） |

返回logit的话到分类还要经过

`torch.nn.functional.softmax`

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

# 1. 加载 tokenizer 和 model
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast=True)
model = AutoModel.from_pretrained(
    checkpoint,
    output_attentions=True,      # 返回注意力权重（可选）
    output_hidden_states=True,   # 返回所有层的隐藏状态（可选）
)

# 2. 输入文本
text = "Hello, how are you today? I am Grok, built by xAI!"

# 3. 使用 tokenizer 预处理文本
inputs = tokenizer(
    text,
    return_tensors="pt",         # 返回 PyTorch 张量
    max_length=20,
    padding="max_length",
    truncation=True,
    return_attention_mask=True,
)

# 4. 将输入传递给模型
outputs = model(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
)

# 5. 输出结果
print("原始文本:", text)
print("\nModel 输出:")
print("最后一层隐藏状态 (last_hidden_state):", outputs.last_hidden_state.shape)
print("池化输出 (pooler_output):", outputs.pooler_output.shape)
print("所有隐藏状态数量:", len(outputs.hidden_states))
print("注意力权重数量:", len(outputs.attentions))

# 6. 批量输入示例
batch_text = ["Hello world!", "I am Grok."]
batch_inputs = tokenizer(
    batch_text,
    padding=True,
    truncation=True,
    max_length=10,
    return_tensors="pt"
)
batch_outputs = model(**batch_inputs)  # 使用 ** 解包字典输入
print("\n批量输入结果:")
print("最后一层隐藏状态:", batch_outputs.last_hidden_state.shape)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


原始文本: Hello, how are you today? I am Grok, built by xAI!

Model 输出:
最后一层隐藏状态 (last_hidden_state): torch.Size([1, 20, 768])
池化输出 (pooler_output): torch.Size([1, 768])
所有隐藏状态数量: 13
注意力权重数量: 12

批量输入结果:
最后一层隐藏状态: torch.Size([2, 7, 768])


bert-base-uncased(110M)

**输入参数**

1. **from_pretrained() 参数**：
    - checkpoint: 指定模型名称（如 "bert-base-uncased"）。
    - cache_dir: 指定缓存路径。
    - output_attentions: 是否返回注意力权重。
    - output_hidden_states: 是否返回所有层的隐藏状态。
2. **model() 参数**：
    - input_ids: 来自 tokenizer 的 token ID 张量。
    - attention_mask: 注意力掩码，区分有效 token 和填充。
    - 可选：token_type_ids（句子对任务）、position_ids 等。

**输出内容**

1. **last_hidden_state**：
    - 形状 [batch_size, sequence_length, hidden_size]。
    - 表示最后一层的隐藏状态，每个 token 有一个 768 维向量（对于 BERT-base）。
2. **pooler_output**：
    - 形状 [batch_size, hidden_size]。
    - [CLS] token 的隐藏状态经过池化层（线性 + tanh）后的输出，常用于分类任务。
3. **hidden_states**（可选）：
    - 一个元组，包含所有层的隐藏状态（包括嵌入层和 12 个 Transformer 层，共 13 个）。
    - 每个形状为 [batch_size, sequence_length, hidden_size]。
4. **attentions**（可选）：
    - 一个元组，包含每层的注意力权重（12 层）。
    - 每个形状为 [batch_size, num_heads, sequence_length, hidden_size]，num_heads=12。

开启 output_hidden_states 或 output_attentions 会**显著增加内存使用**。

**批量输入**

- 输入多个句子时，padding=True 确保长度对齐。
- 输出张量的 batch_size 变为输入样本数（这里是 2）。

**保存模型**：

```python
model.save_pretrained("./my_model")
```
**GPU 支持**

- 使用 .to("cuda") 将模型和输入移动到 GPU。
- 确保 PyTorch 和模型输入都在同一设备上。

**冻结参数**：python

```python
for param in model.base_model.parameters():
    param.requires_grad = False  # 不再更新
```